In [1]:
import pandas as pd
import numpy
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,r2_score
import yfinance as yf

#Used Models
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [2]:
symbol='^NSEI'
ticker = yf.Ticker(symbol)
data=ticker.history(period='max',interval='1m')
data.reset_index(inplace=True)

In [3]:
data.head()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-04-15 09:15:00+05:30,22356.699219,22369.099609,22319.300781,22358.050781,0,0.0,0.0
1,2024-04-15 09:16:00+05:30,22353.599609,22363.650391,22340.949219,22343.349609,0,0.0,0.0
2,2024-04-15 09:17:00+05:30,22341.650391,22344.300781,22325.300781,22329.550781,0,0.0,0.0
3,2024-04-15 09:18:00+05:30,22329.599609,22334.449219,22308.599609,22315.449219,0,0.0,0.0
4,2024-04-15 09:19:00+05:30,22314.750000,22330.449219,22313.750000,22325.199219,0,0.0,0.0


In [4]:
data.tail()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
1495,2024-04-19 15:25:00+05:30,22141.150391,22142.400391,22135.599609,22135.599609,0,0.0,0.0
1496,2024-04-19 15:26:00+05:30,22134.949219,22135.750000,22129.050781,22130.500000,0,0.0,0.0
1497,2024-04-19 15:27:00+05:30,22130.050781,22136.250000,22129.849609,22134.449219,0,0.0,0.0
1498,2024-04-19 15:28:00+05:30,22134.250000,22145.449219,22133.000000,22145.449219,0,0.0,0.0
1499,2024-04-19 15:29:00+05:30,22139.599609,22154.949219,22139.599609,22149.099609,0,0.0,0.0


In [5]:
# data = pd.read_csv('/content/Nifty50_shortTerm_5m_New.csv')

print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype                       
---  ------        --------------  -----                       
 0   Datetime      1500 non-null   datetime64[ns, Asia/Kolkata]
 1   Open          1500 non-null   float64                     
 2   High          1500 non-null   float64                     
 3   Low           1500 non-null   float64                     
 4   Close         1500 non-null   float64                     
 5   Volume        1500 non-null   int64                       
 6   Dividends     1500 non-null   float64                     
 7   Stock Splits  1500 non-null   float64                     
dtypes: datetime64[ns, Asia/Kolkata](1), float64(6), int64(1)
memory usage: 93.9 KB
None


In [6]:
print(data.describe())


               Open          High           Low         Close  Volume  \
count   1500.000000   1500.000000   1500.000000   1500.000000  1500.0   
mean   22172.038221  22176.984383  22166.637143  22172.006557     0.0   
std      151.248187    150.433599    151.814539    151.041816     0.0   
min    21793.750000  21797.500000  21778.550781  21794.400391     0.0   
25%    22113.575195  22119.725586  22107.125000  22113.975098     0.0   
50%    22178.300781  22183.325195  22172.924805  22178.225586     0.0   
75%    22298.112305  22301.287598  22294.262695  22297.937988     0.0   
max    22425.949219  22426.550781  22420.000000  22424.550781     0.0   

       Dividends  Stock Splits  
count     1500.0        1500.0  
mean         0.0           0.0  
std          0.0           0.0  
min          0.0           0.0  
25%          0.0           0.0  
50%          0.0           0.0  
75%          0.0           0.0  
max          0.0           0.0  


In [7]:
data.drop_duplicates()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-04-15 09:15:00+05:30,22356.699219,22369.099609,22319.300781,22358.050781,0,0.0,0.0
1,2024-04-15 09:16:00+05:30,22353.599609,22363.650391,22340.949219,22343.349609,0,0.0,0.0
2,2024-04-15 09:17:00+05:30,22341.650391,22344.300781,22325.300781,22329.550781,0,0.0,0.0
3,2024-04-15 09:18:00+05:30,22329.599609,22334.449219,22308.599609,22315.449219,0,0.0,0.0
4,2024-04-15 09:19:00+05:30,22314.750000,22330.449219,22313.750000,22325.199219,0,0.0,0.0
...,...,...,...,...,...,...,...,...
1495,2024-04-19 15:25:00+05:30,22141.150391,22142.400391,22135.599609,22135.599609,0,0.0,0.0
1496,2024-04-19 15:26:00+05:30,22134.949219,22135.750000,22129.050781,22130.500000,0,0.0,0.0
1497,2024-04-19 15:27:00+05:30,22130.050781,22136.250000,22129.849609,22134.449219,0,0.0,0.0
1498,2024-04-19 15:28:00+05:30,22134.250000,22145.449219,22133.000000,22145.449219,0,0.0,0.0


In [8]:
data.isnull().sum()

Datetime        0
Open            0
High            0
Low             0
Close           0
Volume          0
Dividends       0
Stock Splits    0
dtype: int64

In [9]:
data.drop(['Datetime','Volume','Dividends','Stock Splits'],axis=1,inplace=True)
data.head(10)



,Open,High,Low,Close
0,22356.699219,22369.099609,22319.300781,22358.050781
1,22353.599609,22363.650391,22340.949219,22343.349609
2,22341.650391,22344.300781,22325.300781,22329.550781
3,22329.599609,22334.449219,22308.599609,22315.449219
4,22314.750000,22330.449219,22313.750000,22325.199219
5,22324.199219,22334.949219,22319.949219,22319.949219
6,22320.050781,22323.300781,22302.800781,22312.699219
7,22312.800781,22349.699219,22312.800781,22349.349609
8,22349.900391,22356.150391,22345.500000,22351.949219
9,22352.250000,22352.250000,22324.250000,22324.250000


In [10]:
data['shift_close'] = data['Close'].shift(-1)
data.dropna(inplace=True)

In [11]:
data

,Open,High,Low,Close,shift_close
0,22356.699219,22369.099609,22319.300781,22358.050781,22343.349609
1,22353.599609,22363.650391,22340.949219,22343.349609,22329.550781
2,22341.650391,22344.300781,22325.300781,22329.550781,22315.449219
3,22329.599609,22334.449219,22308.599609,22315.449219,22325.199219
4,22314.750000,22330.449219,22313.750000,22325.199219,22319.949219
...,...,...,...,...,...
1494,22140.750000,22143.599609,22134.099609,22142.050781,22135.599609
1495,22141.150391,22142.400391,22135.599609,22135.599609,22130.500000
1496,22134.949219,22135.750000,22129.050781,22130.500000,22134.449219
1497,22130.050781,22136.250000,22129.849609,22134.449219,22145.449219


In [12]:
data.tail()

,Open,High,Low,Close,shift_close
1494,22140.750000,22143.599609,22134.099609,22142.050781,22135.599609
1495,22141.150391,22142.400391,22135.599609,22135.599609,22130.500000
1496,22134.949219,22135.750000,22129.050781,22130.500000,22134.449219
1497,22130.050781,22136.250000,22129.849609,22134.449219,22145.449219
1498,22134.250000,22145.449219,22133.000000,22145.449219,22149.099609


In [13]:
X=data.drop('shift_close',axis=1)
y = data['shift_close']

In [14]:
xy=pd.concat([X,y],axis=1)
xy.tail()

,Open,High,Low,Close,shift_close
1494,22140.750000,22143.599609,22134.099609,22142.050781,22135.599609
1495,22141.150391,22142.400391,22135.599609,22135.599609,22130.500000
1496,22134.949219,22135.750000,22129.050781,22130.500000,22134.449219
1497,22130.050781,22136.250000,22129.849609,22134.449219,22145.449219
1498,22134.250000,22145.449219,22133.000000,22145.449219,22149.099609


In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [16]:

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

predictions = my_model.predict(X_test)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_test)))

print("R2 Score" + str(r2_score(y_test,predictions)))


print("Accuracy Score : "+str(my_model.score(X_test,y_test)))

predictions


Mean Absolute Error: 9.138763020833334
R2 Score0.9828355862031668
Accuracy Score : 0.9828355862031668


array([21891.68 , 22323.941, 22295.76 , 22349.6  , 22113.361, 22034.314,
       22033.852, 22400.877, 22170.516, 22336.453, 22402.559, 22158.389,
       22113.246, 22301.277, 22198.008, 22194.373, 22142.951, 22384.146,
       22290.102, 22208.43 , 22152.275, 22194.941, 22373.242, 22158.281,
       22119.4  , 22154.457, 22157.123, 22145.781, 22140.219, 22033.117,
       22003.492, 22126.64 , 22193.562, 22031.113, 22363.873, 22266.041,
       22149.385, 22164.205, 22195.42 , 22294.432, 21881.98 , 22143.562,
       22042.99 , 22395.598, 22078.322, 21884.033, 22116.303, 22314.582,
       22293.121, 22142.93 , 22187.666, 22377.604, 22072.451, 22124.895,
       22334.988, 22305.188, 21891.68 , 21856.01 , 22196.01 , 22042.482,
       22300.824, 22126.283, 22181.84 , 21862.492, 21921.172, 22332.29 ,
       22192.926, 21908.807, 22142.203, 22204.168, 22288.41 , 22164.533,
       22152.006, 21965.223, 22383.625, 22139.145, 21855.326, 21839.46 ,
       22374.303, 21922.902, 22123.076, 22346.045, 

In [17]:
RandomeForestModel = RandomForestRegressor()
RandomeForestModel.fit(X_train, y_train)


predictions = RandomeForestModel.predict(X_test)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_test)))

print("R2 Score" + str(r2_score(y_test,predictions)))


print("Accuracy Score : "+str(RandomeForestModel.score(X_test,y_test)))

predictions

Mean Absolute Error: 8.222180924479144
R2 Score0.9849583619820679
Accuracy Score : 0.9849583619820679


array([21889.04201172, 22328.43724609, 22295.26875   , 22352.41015625,
       22110.95482422, 22031.80626953, 22025.63783203, 22399.49050781,
       22178.4009375 , 22337.67964844, 22399.63      , 22159.93041016,
       22113.49025391, 22304.24755859, 22195.80152344, 22193.6715625 ,
       22145.73880859, 22379.77054687, 22288.64099609, 22211.79695312,
       22152.43833984, 22197.23408203, 22370.46744141, 22161.73746094,
       22122.62328125, 22155.34138672, 22151.63056641, 22144.59210938,
       22140.84212891, 22037.14984375, 22004.58677734, 22129.62273437,
       22195.09210938, 22038.05845703, 22362.39691406, 22264.23431641,
       22146.73619141, 22153.53523437, 22195.08427734, 22292.89287109,
       21876.93234375, 22144.7571875 , 22046.07603516, 22396.87671875,
       22084.81970703, 21885.03386719, 22119.20935547, 22308.83660156,
       22294.25779297, 22143.86458984, 22191.80041016, 22375.07847656,
       22088.94521484, 22125.99712891, 22333.29759766, 22303.94527344,
      

In [18]:
DesitionTreeModel = DecisionTreeRegressor()
DesitionTreeModel.fit(X_train, y_train)


predictions = DesitionTreeModel.predict(X_test)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_test)))

print("R2 Score" + str(r2_score(y_test,predictions)))


print("Accuracy Score : "+str(DesitionTreeModel.score(X_test,y_test)))

predictions

Mean Absolute Error: 10.2016015625
R2 Score0.9788856990661915
Accuracy Score : 0.9788856990661915


array([21889.        , 22327.90039062, 22300.69921875, 22353.80078125,
       22119.80078125, 22031.25      , 22011.65039062, 22404.75      ,
       22177.44921875, 22337.        , 22400.        , 22167.90039062,
       22119.80078125, 22301.        , 22192.69921875, 22191.5       ,
       22154.90039062, 22380.25      , 22286.40039062, 22209.59960938,
       22148.94921875, 22193.09960938, 22371.84960938, 22155.09960938,
       22104.90039062, 22154.69921875, 22150.59960938, 22140.15039062,
       22140.40039062, 22034.44921875, 22012.15039062, 22133.05078125,
       22196.19921875, 22046.15039062, 22362.09960938, 22278.55078125,
       22140.15039062, 22172.59960938, 22197.59960938, 22291.5       ,
       21882.55078125, 22140.15039062, 22044.75      , 22394.25      ,
       22090.25      , 21884.94921875, 22120.34960938, 22314.65039062,
       22294.09960938, 22154.90039062, 22188.55078125, 22382.34960938,
       22094.59960938, 22122.40039062, 22344.59960938, 22307.34960938,
      

In [19]:

import pickle
pickle.dump(my_model,open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/XGBoost.pkl','wb'))
pickle.dump(RandomeForestModel,open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/RandomForestRegressor.pkl','wb'))
pickle.dump(DesitionTreeModel,open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/DesicionTreeModel.pkl','wb'))

# pickle.dump(my_model,open('/content/drive/MyDrive/AI COLLAGE/5m_Nifty_RandomForestRegressor.pkl','wb'))


import yfinance as yf
import datetime as dt
import pickle
from sklearn.metrics import mean_absolute_error,r2_score,accuracy


# Load the model from the pickle file
with open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/RandomForestRegressor.pkl', 'rb') as f:
    Rand = pickle.load(f)


with open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/DesicionTreeModel.pkl', 'rb') as f:
    Decision = pickle.load(f)


with open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/XGBoost.pkl', 'rb') as f:
    XgBoostModel = pickle.load(f)



while True:
    ticker = yf.Ticker('^NSEI')
    start=dt.datetime.now().strftime('%Y-%m-%d')
    end=(dt.datetime.now() + dt.timedelta(days=1)).strftime('%Y-%m-%d')

    data = ticker.history(interval='5m',start=start,end=end)
    data.reset_index(inplace=True)
    # print(data)
    data.columns = data.columns.str.strip()  # Remove extra spaces from column names

    data.drop(['Datetime', 'Volume', 'Dividends', 'Stock Splits'], inplace=True, axis=1)
    y = data['Close'].copy()
    data.drop(['Close'], inplace=True, axis=1)

    print("Random Forest Regressor")
    print("---------------------------\n")
    Rand_predic=Rand.predict(data)
    last_pred=Rand_predic[-1]
    print(f'Last Prediction ----------> :{ last_pred}')
    print(f'mae : {mean_absolute_error(y,Rand_predic)}')
    # print(f"Rand Accuracy Score :{}")
    print(f'r2 : {r2_score(y,Rand_predic)}')
    print("---------------------------")
    print("\n\n\n")

    print("Desicion Tree Regressor")
    print("---------------------------\n")
    DecisionPred=Decision.predict(data)
    last_pred=DecisionPred[-1]
    mae=mean_absolute_error(y,DecisionPred)
    # last_pred=last_pred+mae
    print(f'Last Prediction ----------> :{ last_pred}')
    print(f'mae : {mae}')
    print(f'r2 : {r2_score(y,DecisionPred)}')
    print("---------------------------")
    print("\n\n\n")

    print("XgBoostModel ")
    print("---------------------------\n")
    XgBoostPred=XgBoostModel.predict(data)
    last_pred=XgBoostPred[-1]
    print(f'Last Prediction ----------> :{ last_pred}')
    print(f'mae : {mean_absolute_error(y,XgBoostPred)}')
    print(f'r2 : {r2_score(y,XgBoostPred)}')
    print("---------------------------")
    print("\n\n\n")
    # data